## LSTM

长短期记忆 (LSTM) 是一种循环神经网络架构，由 Sepp Hochreiter 和 Jürgen Schmidhuber 于 1997 年设计。LSTM 架构由一个单元组成，即记忆单元（也称为 LSTM 单元）， LSTM 单元由四个前馈神经网络组成，每个神经网络都由输入层和输出层组成。在每个神经网络中，输入神经元都连接到所有输出神经元。因此，LSTM 单元有四个全连接层。

四个前馈神经网络中的三个负责选择信息。它们是遗忘门、输入门和输出门。这三个门用于执行三种典型的内存管理操作：从内存中删除信息（遗忘门）、在内存中插入新信息（输入门）以及使用内存中存在的信息（输出门）。第四个神经网络，即候选存储器，用于创建要插入到存储器中的新候选信息。

![LSTM](images/LSTM.png)

### 输入和输出

LSTM 单元接收三个向量（三个数字列表）作为输入。两个向量来自 LSTM 本身，并由 LSTM 在前一时刻（时刻$t − 1$）生成。它们是单元状态 ( $C$ ) 和隐藏状态 ( $H$ )。第三个向量来自外部。这是在时刻$t$提交给 LSTM 的向量$X$ （称为输入向量）。


给定三个输入向量（ $C$ 、 $H$ 、 $X$ ），LSTM 通过门调节内部信息流并转换单元状态和隐藏状态向量的值。将成为下一时刻（时刻$t + 1$）的 LSTM 输入集一部分的向量。进行信息流控制，使单元状态充当长期记忆，而隐藏状态充当短期记忆。

实际上，LSTM 单元使用最近的过去信息（短期记忆， $H$ ）和来自外部的新信息（输入向量， $X$ ）来更新长期记忆（单元状态， $C$ ）。最后，它使用长期记忆（细胞状态， $C$ ）来更新短期记忆（隐藏状态， $H$ ）。在时刻$t$确定的隐藏状态也是 LSTM 单元在时刻$t$的输出，这是 LSTM 向外部提供的用于执行特定任务的内容。换句话说，它是评估 LSTM 性能的行为。

### 门

三个门（遗忘门、输入门和输出门）是信息选择器，他们的任务是创建选择器向量，选择器向量是值介于 0 和 1 之间且接近这两个极值的向量。

创建一个选择器向量，将其逐个元素地与另一个相同大小的向量相乘。这意味着选择器向量具有等于0的值的位置完全消除了（在逐个元素相乘中）包括在另一向量中的相同位置中的信息。选择器向量具有等于一的值的位置使包括在另一向量中的相同位置中的信息保持不变（在逐个元素相乘中）。

所有三个门都是使用 sigmoid 函数作为输出层激活函数的神经网络。 sigmoid 函数用于将由 0 到 1 之间以及接近这两个极值的值组成的向量作为输出。

所有三个门都使用输入向量 ( $X$ ) 和来自前一时刻 ( $H_{[t−1]}$ ) 的隐藏状态向量，并将其连接在一个向量中，该向量是所有三个门的输入。




## LSTM的实现
LSTM 和基本的 RNN 是一样的，他的参数也是相同的，同时他也有 `nn.LSTMCell()` 和 `nn.LSTM()` 两种形式，跟前面讲的都是相同的，我们就不再赘述了，下面直接举个小例子

In [58]:
lstm_seq = nn.LSTM(50, 100, num_layers=2) # 输入维度 100，输出 200，两层

In [80]:
lstm_seq.weight_hh_l0 # 第一层的 h_t 权重

Parameter containing:
1.00000e-02 *
 3.8420  5.7387  6.1351  ...   1.2680  0.9890  1.3037
-4.2301  6.8294 -4.8627  ...  -6.4147  4.3015  8.4103
 9.4411  5.0195  9.8620  ...  -1.6096  9.2516 -0.6941
          ...             ⋱             ...          
 1.2930 -1.3300 -0.9311  ...  -6.0891 -0.7164  3.9578
 9.0435  2.4674  9.4107  ...  -3.3822 -3.9773 -3.0685
-4.2039 -8.2992 -3.3605  ...   2.2875  8.2163 -9.3277
[torch.FloatTensor of size 400x100]

**小练习：想想为什么这个系数的大小是 (400, 100)**

In [59]:
lstm_input = torch.randn(10, 3, 50) # 序列 10，batch 是 3，输入维度 50

In [64]:
out, (h, c) = lstm_seq(lstm_input) # 使用默认的全 0 隐藏状态

注意这里 LSTM 输出的隐藏状态有两个，h 和 c，就是上图中的每个 cell 之间的两个箭头，这两个隐藏状态的大小都是相同的，(num_layers * direction, batch, feature)

In [66]:
h.shape # 两层，Batch 是 3，特征是 100

torch.Size([2, 3, 100])

In [67]:
c.shape

torch.Size([2, 3, 100])

In [61]:
out.shape

torch.Size([10, 3, 100])

我们可以不使用默认的隐藏状态，这是需要传入两个张量

In [68]:
h_init = Variable(torch.randn(2, 3, 100))
c_init = Variable(torch.randn(2, 3, 100))

In [69]:
out, (h, c) = lstm_seq(lstm_input, (h_init, c_init))

In [70]:
h.shape

torch.Size([2, 3, 100])

In [71]:
c.shape

torch.Size([2, 3, 100])

In [72]:
out.shape

torch.Size([10, 3, 100])

## GRU


门控循环单元 (GRU) 是循环神经网络 (RNN) 的一种，在某些情况下，比长短期记忆 (LSTM) 具有优势。 GRU 使用更少的内存并且比 LSTM 更快，但是，当使用具有较长序列的数据集时，LSTM 更准确。

此外，GRU 还解决了普通循环神经网络所面临的梯度消失问题（用于更新网络权重的值）。如果评分在反向传播时随着时间的推移而缩小，它可能会变得太小而无法影响学习，从而使神经网络无法训练。

如果神经网络中的层无法学习，RNN 基本上会“忘记”更长的序列。

GRU 通过使用两个门（更新门和重置门）来解决这个问题。这些门决定哪些信息被允许通过输出，并且可以被训练以保留更远的信息。这使得它能够将相关信息传递到一系列事件中，以做出更好的预测。


In [73]:
gru_seq = nn.GRU(10, 20)
gru_input = torch.randn(3, 32, 10)

out, h = gru_seq(gru_input)

In [76]:
gru_seq.weight_hh_l0

Parameter containing:
 0.0766 -0.0548 -0.2008  ...  -0.0250 -0.1819  0.1453
-0.1676  0.1622  0.0417  ...   0.1905 -0.0071 -0.1038
 0.0444 -0.1516  0.2194  ...  -0.0009  0.0771  0.0476
          ...             ⋱             ...          
 0.1698 -0.1707  0.0340  ...  -0.1315  0.1278  0.0946
 0.1936  0.1369 -0.0694  ...  -0.0667  0.0429  0.1322
 0.0870 -0.1884  0.1732  ...  -0.1423 -0.1723  0.2147
[torch.FloatTensor of size 60x20]

In [75]:
h.shape

torch.Size([1, 32, 20])

In [74]:
out.shape

torch.Size([3, 32, 20])

## 参考资料
* [An Intuitive Explanation of LSTM](https://medium.com/@ottaviocalzone/an-intuitive-explanation-of-lstm-a035eb6ab42c)